# HW 7

## Question 1

Here is a table of 1-5 star ratings for five movies (M, N, P. Q. R) by three raters (A, B, C). Normalize the ratings by subtracting the average for each row and then subtracting the average for each column in the resulting table. Then, identify the true statement about the normalized table.

**Solution**

In [57]:
import pandas as pd


A = [
    [1,2,3,4,5],
    [2,3,2,5,3],
    [5,5,5,3,2]
]

A = pd.DataFrame(A, index=["A", "B", "C"], columns=["M", "N", "P", "Q", "R"])
A = A.subtract(A.mean(axis=1), axis=0)
A = A.subtract(A.mean(), axis=1)
A.apply(lambda x: round(x, 2))

,M,N,P,Q,R
A,-1.33,-1.0,0.0,0.33,2.0
B,-0.33,0.0,-1.0,1.33,0.0
C,1.67,1.0,1.0,-1.67,-2.0


## Question 2

Below is a table giving the profile of three items.

The first five attributes are Boolean, and the last is an integer "rating." Assume that the scale factor for the rating is α. Compute, as a function of α, the cosine distances between each pair of profiles. For each of α = 0, 0.5, 1, and 2, determine the cosine of the angle between each pair of vectors. Which of the following is FALSE?

**Solution**

In [31]:
S = [
    [1,0,1,0,1,2],
    [1,1,0,0,1,6],
    [0,1,0,1,0,2]
]

labels = ["A", "B", "C"]

for alpha in [0, .5, 1, 2]:
    print("alpha =", alpha)
    for i, j in [(0, 1), (0, 2), (1, 2)]:
        x = S[i]
        y = S[j]
        d = sum(x[k] * y[k] for k in range(5)) + alpha**2 * x[5] * y[5]
        u = (sum(c**2 for c in x[:5]) + alpha**2 * x[5]**2)**.5
        v = (sum(c**2 for c in y[:5]) + alpha**2 * y[5]**2)**.5
        cos = d / (u * v)
        print ("(%s, %s) cos similarity is %s" % (labels[i], labels[j], round(cos, 3)))

alpha = 0
(A, B) cos similarity is 0.667
(A, C) cos similarity is 0.0
(B, C) cos similarity is 0.408
alpha = 0.5
(A, B) cos similarity is 0.722
(A, C) cos similarity is 0.289
(B, C) cos similarity is 0.667
alpha = 1
(A, B) cos similarity is 0.847
(A, C) cos similarity is 0.617
(B, C) cos similarity is 0.85
alpha = 2
(A, B) cos similarity is 0.946
(A, C) cos similarity is 0.865
(B, C) cos similarity is 0.953


## Question 3

Below is a utility matrix representing ratings by users A, B, and C for items a through h.

Treat ratings of 3, 4, and 5 as 1 and 1, 2, and blank as 0. Compute the Jaccard distance between each pair of items. Then, cluster the items hierarchically into four clusters, using the Jaccard distance. When a cluster consists of more than one item, take the distance between clusters to be the minimum over all pairs of items, one from each cluster, of the Jaccard distance between those items. Break ties lexicographically. That is, sort the items that would be merged alphabetically, and merge those clusters whose resulting set would be first alphabetically. Then, identify one of the resulting clusters in the list below.
Note: if you are not familiar with hierarchical clustering, you might want to wait until Video 59 in Week 5, or read Sect. 7.2 of the MMDS book.

**Solution**

In [61]:
items = {
    "a": (4, 0, 2),
    "b": (5, 3, 0),
    "c": (0, 4, 1),
    "d": (5, 3, 3),
    "e": (1, 1, 0),
    "f": (0, 2, 4),
    "g": (3, 1, 5),
    "h": (2, 0, 3)
}

for c, v in items.items():
    items[c] = tuple(1 if k >= 3 else 0 for k in v)

clusters = list(map(lambda c: {c}, items.keys()))

def jac_dist(a, b):
    ia = items[a]
    ib = items[b]
    dist = sum(x & y for x, y in zip(ia, ib)) / sum(x | y for x, y in zip(ia, ib))
    return 1 - dist

def clust_dist(i, j):
    
    dist = float("inf")
    
    for x in clusters[i]:
        for y in clusters[j]:
            dist = min(dist, jac_dist(x, y))
    
    return dist

while len(clusters) > 4:
    
    count = len(clusters)
    
    min_dist = min(
        clust_dist(i, j)
        for i in range(count - 1)
        for j in range(i + 1, count)
    )
    
    candidates = [
        (i, j)
        for i in range(count - 1)
        for j in range(i + 1, count)
        if clust_dist(i, j) == min_dist
    ]
    
    x, y = min(
        candidates,
        key = lambda c: ''.join(sorted(clusters[c[0]] | clusters[c[1]]))
    )
    
    c1 = clusters.pop(max(x, y))
    c2 = clusters.pop(min(x, y))
    clusters.append(c1 | c2)
    
print(clusters)        

[{'c'}, {'e'}, {'h', 'f'}, {'b', 'g', 'd', 'a'}]
